### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
# print the data head to get a basic idea of the data structure
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [2]:
# Players are listed in the "SN" column, we need to find the unique names and get the total number of players
playernum = len(purchase_data["SN"].unique())
total_players = pd.DataFrame({"Total Players" : playernum}, index = [0])

# show total players
total_players

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
# for this part, we need to analyse "Item Name", "Price" columns, 
# and need to count the total purchases

total_purchase = len(purchase_data)
unique_items = len(purchase_data["Item Name"].unique())
total_revenue = purchase_data["Price"].sum()
avg_price = purchase_data["Price"].mean() #total_revenue/total_purchase

purchasing_analysis = pd.DataFrame({
                                    "Number of Unique Items" : [unique_items],
                                    "Average Price" : [avg_price],
                                    "Number of Purchases" : [total_purchase],
                                    "Total Revenue" : [total_revenue]
                                    })
# convert to currency for specific columns
purchasing_analysis["Average Price"] = purchasing_analysis["Average Price"].map('${:,.2f}'.format)
purchasing_analysis["Total Revenue"] = purchasing_analysis["Total Revenue"].map('${:,.2f}'.format)

# show summary data frame
purchasing_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
# count male players and calculate percentage
males = len(purchase_data.loc[purchase_data['Gender'] == "Male", "SN"].unique())
percent_males = str(round(males / playernum * 100, 2)) + "%"
# count female players and calculate percentage
females = len(purchase_data.loc[purchase_data['Gender'] == "Female", "SN"].unique())
percent_females = str(round(females / playernum * 100, 2)) + "%"
# count non-specified players and calcualte percentage
other = len(purchase_data.loc[purchase_data['Gender'] == "Other / Non-Disclosed", "SN"].unique())
percent_other = str(round(other / playernum * 100, 2)) + "%"

GD = pd.DataFrame({
                    "Total Count": [males, females, other],
                    "Percentage of Players": [percent_males, percent_females, percent_other]
}, index = ["Male", "Female", "Other/Non-Disclosed"])

# Show summary
GD

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other/Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
# define a function to calculate the results
# I didn't use groupby function, using loc function instead, just for practice, 
# even though the rubic said we need to use groupby fuction to earn points for this part
def purchasing_analysis_gender(gender, gender_count):
    gender = gender
    count = len(purchase_data["Gender"][purchase_data['Gender'] == gender])
    tol_pvalue = purchase_data.loc[purchase_data['Gender'] == gender, "Price"].sum()
    avg_p = purchase_data.loc[purchase_data['Gender'] == gender, "Price"].mean() # I think use tol_pvalue/count is better, 
                                                                                 # but since the rubic need us to use mean function, so...
    avg_p_perperson = tol_pvalue/gender_count
    return gender, count, avg_p, tol_pvalue,avg_p_perperson

# Calling function
results_male = purchasing_analysis_gender("Male", males)
results_female = purchasing_analysis_gender("Female", females)
results_other = purchasing_analysis_gender("Other / Non-Disclosed", other)

# merge results returned from the function for each gender by zip
results = zip(results_female,results_male, results_other)

# transpose the results to generate summary DataFrame
results_T = map(list, zip(*results))

# create the data frame and map columns to get currency expression
Purchase_Analysis_Gender = pd.DataFrame(results_T, columns = ["Gender", "Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"])
Purchase_Analysis_Gender["Average Purchase Price"] = Purchase_Analysis_Gender["Average Purchase Price"].map("${:,.2f}".format)
Purchase_Analysis_Gender["Total Purchase Value"] = Purchase_Analysis_Gender["Total Purchase Value"].map("${:,.2f}".format)
Purchase_Analysis_Gender["Avg Total Purchase per Person"] = Purchase_Analysis_Gender["Avg Total Purchase per Person"].map("${:,.2f}".format)

# set index use the first column of the data frame
Purchase_Analysis_Gender = Purchase_Analysis_Gender.set_index("Gender")

# show the summary table
Purchase_Analysis_Gender

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
# create bin for ages
# when cutting the data by bins, I found out by default it include the left but exclude the right boundary, 
# so I set bin as 0,9, 14... instead of 0,10,15...
# also set 39-100 for age interval bigger than 40 as normally one can't go beyound 100
bin_age = [0, 9, 14, 19, 24, 29, 34, 39, 100]
catg_age = pd.cut(purchase_data["Age"], bin_age)

# make a copy of raw purchase data
purchase_data_copy = purchase_data
purchase_data_copy["catg_age"] = catg_age # update copied data frame by adding the age interval column

# define a function to calculate the results
def age_demographics():
    categories = sorted(purchase_data_copy["catg_age"].unique()) # sort the age interval from low to high
    combined_data = purchase_data_copy.groupby('catg_age') # group dataframe by age intervals
    unique_player_catg = combined_data["SN"].unique() # get a list of unique players for each age interval
    num_of_players = [] # create a blank list to store player numbers for each age interval
    percent = [] # create a blank list to store each percentage
    # calculate player numbers and percentages for each intercal
    for i in categories:
        num_of_players.append(len(unique_player_catg[i]))
        percent.append(str(round(len(unique_player_catg[i])/playernum*100, 2))+"%")
    return num_of_players, percent

# create nested list to generate final dataframe
results = age_demographics()
results_T = map(list, zip(*results)) # Transpose the results for dataframe generation

AD = pd.DataFrame(results_T, columns = ["Total Count", "Percentage of Players"],
                             index = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"])

# show summary data frame
AD

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
# define a function to calculate the results
def Purchasing_Analysis_Age():
    '''
    for this part, I can use pd.sort_values on the groupby'ed data frame by age interval ascendingly, then do sum,
    mean, count and etc on the sorted groupby'ed data frame
    but here I used a for loop instead, just to refresh my mind of other ways to sovle the problem
    still, as we can see the following cells of codes using groupby, less lines of coding, groupby is better
    '''
    categories = sorted(purchase_data_copy["catg_age"].unique()) # sort the age interval from low to high
    combined_data = purchase_data_copy.groupby('catg_age') # group dataframe by age intervals
    total_num_purchase = combined_data.count()['Purchase ID'] # a list of purchase count for all age intervals
    unique_player_catg = combined_data["SN"].unique() # get a list of unique players for each age interval
    age_interval = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
    # create so blank lists to store data
    total_count = []
    avg_price = []
    total_value = []
    avg_price_person = []
    # calculate results for each age range
    for i in categories:
        # list of total number of purchases per age range
        total_count.append(total_num_purchase[i])
        # list of total purchase value per age range
        total_value.append("$"+str(round(combined_data.sum()["Price"][i],2)))
        # average price per age range
        avg_price.append("$"+str(round(combined_data.sum()["Price"][i]/total_num_purchase[i],2)))
        # average price per person per age range
        avg_price_person.append("$"+ str(round(combined_data.sum()["Price"][i]/len(unique_player_catg[i]),2)))
    return age_interval, total_count, avg_price, total_value, avg_price_person

# get each returned value seperately and then combine them into a list, get the transpose
results = Purchasing_Analysis_Age()
results_T = map(list, zip(*results)) # Transpose the results for dataframe generation

# add columns and use the first column as index
PAG = pd.DataFrame(results_T, 
                   columns = ["Age Ranges", "Purchase Count", "Average Purchase Price", "Total Purchase Value","Avg Total Purchase per Person"],
                  ).set_index('Age Ranges')

# show summary data frame
PAG

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.9,$293.0,$3.81
30-34,73,$2.93,$214.0,$4.12
35-39,41,$3.6,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
# define a function definition to calculate the results
def Top_Spenders():
    # group by players names
    player_group = purchase_data.groupby('SN')
    tol_value = player_group["Price"].sum() # total value for each player spent
    avg_value = player_group["Price"].mean() # average value for each player spent 
                                             # initially used (tol_value/player_group["Price"].count())
    purchase_count = player_group["Price"].count() # number of purchases for each player
    name = player_group["SN"].first() # get the names of players
    return name, purchase_count, avg_value, tol_value

results = Top_Spenders()
results_T = map(list, zip(*results)) # Transpose the results for dataframe generation
TS = pd.DataFrame(results_T, columns = ["SN", "Purchase Count", "Average Purchase Price", "Total Purchase Value"],
                 ).set_index("SN").sort_values(["Total Purchase Value"], ascending=False)

# convert the currency formatting
TS["Average Purchase Price"]=TS["Average Purchase Price"].map("${:,.2f}".format)
TS["Total Purchase Value"]=TS["Total Purchase Value"].map("${:,.2f}".format)

# show a preview of the summary data frame
TS.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
# create a new data frame for item info only
items_data = purchase_data[["Item ID", "Item Name", "Price"]]

# group by Item ID and name
item_grouped = items_data.groupby(['Item ID','Item Name'])

# Function definition to calculate all the results
def Popular_Items():
    num_purchase = item_grouped['Item ID'].count()
    item_price = item_grouped["Price"].first()
    total_value = num_purchase*item_price
    return item_grouped["Item ID"].first(), item_grouped["Item Name"].first(), num_purchase, item_price, total_value

results = Popular_Items()
results_T = map(list, zip(*results)) # Transpose the results for dataframe generation
PI = pd.DataFrame(results_T, columns = ["Item ID", "Item Name", "Purchase Count", "Item Price", "Total Purchase Value"],
                 ).set_index(["Item ID", "Item Name"])

# sort data frame by Purchase Count, descending
PI_PC_sort = PI.sort_values(["Purchase Count"], ascending = False)

# convert to currency formatting
PI_PC_sort["Item Price"] = PI_PC_sort["Item Price"].map("${:,.2f}".format)
PI_PC_sort["Total Purchase Value"] = PI_PC_sort["Total Purchase Value"].map("${:,.2f}".format)

# show a preview of summary data farme
PI_PC_sort.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [10]:
# sort the data frame by Total Purchase Value, descending
PI_TPV_sort = PI.sort_values(["Total Purchase Value"], ascending = False)

# convert to currency formatting
PI_TPV_sort["Item Price"] = PI_TPV_sort["Item Price"].map("${:,.2f}".format)
PI_TPV_sort["Total Purchase Value"] = PI_TPV_sort["Total Purchase Value"].map("${:,.2f}".format)

# show a preview of summary data farme
PI_TPV_sort.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


# Three conclusions from the data for Pymoli

1. In general, there are more male players than female players. males almost dominate the game with a high percentage of around 84%.
2. Even though there are much more males than females, females spent more than males in terms of average expense per person. (males 4.07 dollars while females 4.47 lollars per person)
3. Amoung these players, yonger people aged from 15 - 24 seem to be more attracted to the game. More than half of the players, 62% exactly, are youth and college students. And they buy more items due to their strong curiosity and the desire of showing off to others.
4. Most popular items might not necessarily be the most profitable items as seen from the last 2 summary tables